In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
from datetime import date, datetime
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
sns.set_theme()
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)

# column_names = ['# Seen', 'ALSA', '# Picked', 'ATA', '# GP', 'GP WR', '# OH', 'OH WR', '# GD', 'GD WR', '# GIH', 'GIH WR', '# GND', 'GND WR', 'IWD', 'Color', 'Rarity']

from Logger import Logger
from Fetcher import Fetcher
import WUBRG
import consts
import settings
from LineColors import LineColors

from JSONHandler import JSONHandler
from FormatMetadata import FormatMetadata
from RawDataFetcher import RawDataFetcher
from RawDataHandler import RawDataHandler
from FramedData import FramedData

from CallScryfall import CallScryfall
from CARD import Card
from FormatMetadata import SETS, FORMATS, SET_CONFIG, SetMetadata, FormatMetadata

LOG_LEVEL = Logger.FLG.DEFAULT
LOG_LEVEL

In [ ]:
settings.ROLL

# Objects

In [ ]:
class FramedDataFuncs:   
    def __init__(self, SET, FORMAT):
        self._DATA = FramedData(SET, FORMAT)
        self._ARCH_FUNCS = ArchFuncs(self._DATA)
        self._SNGL_CARD_FUNCS = SingleCardFuncs(self._DATA)
    
    @property
    def SET(self):
        """The draft set."""
        return self._DATA.SET
    
    @property
    def FULL_SET(self):
        """The full name of the draft set."""
        ## TODO: Have this information be pulle from somewhere
        return self._DATA.FULL_SET
    
    @property
    def FORMAT(self):
        """The format type."""
        return self._DATA.FORMAT
    
    @property
    def SHORT_FORMAT(self):
        """The shorthand of the format type."""
        return self._DATA.FULL_FORMAT
    
    @property
    def DATA(self):
        """The object which contains the data about the set and format."""
        return self._DATA
        
    def check_for_updates(self):
        """Populates and updates all data properties, filling in missing data."""
        self._DATA.check_for_updates()

    def reload_data(self):
        """Populates and updates all data properties, reloading all data."""
        self._DATA.reload_data()  
    
    def deck_group_frame(self, name=None, date=None, summary=False):
        """Returns a subset of the 'GROUPED_ARCHTYPE' data as a DataFrame."""
        return self.DATA.deck_group_frame(name, date, summary)
     
    def deck_archetype_frame(self, deck_color=None, date=None, summary=False):
        """Returns a subset of the 'SINGLE_ARCHTYPE' data as a DataFrame."""
        return self.DATA.deck_archetype_frame(deck_color, date, summary)
    
    def card_frame(self, name=None, deck_color=None, date=None, card_color=None, card_rarity=None, summary=False):
        """Returns a subset of the 'CARD' data as a DataFrame."""
        return self.DATA.card_frame(name, deck_color, date, card_color, card_rarity, summary)
    
    def compress_date_range_data(self, start_date, end_date, card_name=None):
        """Summarizes card data over a provided set of time."""
        return self.DATA.compress_date_range_data(start_date, end_date, card_name)
        
    
    #ArchFuncs
    def get_games_played(self, deck_color):
        return self._ARCH_FUNCS.get_games_played(deck_color)
        
    def get_avg_winrate(self, day=None, arch='All Decks'):
        return self._ARCH_FUNCS.get_avg_winrate(day, arch)
    
    def get_archetype_frame(self, colors, roll=None):
        return self._ARCH_FUNCS.get_archetype_frame(colors, roll)

    def get_archetype_winrate_history(self, color_filter=None, roll=None):
        return self._ARCH_FUNCS.get_archetype_winrate_history(color_filter, roll)

    def plot_archetype_winrate_history(self, color_filter=None, roll=None):
        return self._ARCH_FUNCS.plot_archetype_winrate_history(color_filter, roll)       
        
    def get_archetype_playrate_history(self, color_filter=None, roll=None):
        return self._ARCH_FUNCS.get_archetype_playrate_history(color_filter, roll)

    def plot_archetype_playrate_history(self, color_filter=None, roll=None):
        return self._ARCH_FUNCS.plot_archetype_playrate_history(color_filter, roll)
    
    
    #SingleCardFuncs
    def get_card_summary(self, card_name, colors='', roll=None):
        return self._SNGL_CARD_FUNCS.get_card_summary(card_name, colors, roll)

    def plot_card_summary(self, card_name, colors='', roll=None):
        return self._SNGL_CARD_FUNCS.plot_card_summary(card_name, colors, roll)
    
    def get_pick_stats(self, card_name, roll=None):
        return self._SNGL_CARD_FUNCS.get_pick_stats(card_name, roll)
    
    def plot_pick_stats(self, card_name, roll=None):
        return self._SNGL_CARD_FUNCS.plot_pick_stats(card_name, roll)
    
    def card_archetype_performance(self, card_name):
        return self._SNGL_CARD_FUNCS.card_archetype_performance(card_name)
    
    def stat_archetype_performance(self, stat_name, color_cols=None, min_colors=0, max_colors=5):
        return self._SNGL_CARD_FUNCS.stat_archetype_performance(stat_name, color_cols, min_colors, max_colors)

    
    
    def compare_card_evaluations(self, start_date, end_date):
        def inner_func(date):
            df = self.card_frame(date=date, deck_color='')
            df.index = [tup[2] for tup in df.index]
            return df

        first = inner_func(date=start_date)
        last = inner_func(date=end_date)
        diff = last[['ALSA', 'ATA', 'Color', 'Rarity']].copy()
        diff['Δ ALSA'] = first['ALSA'] - last['ALSA']
        diff['Δ ATA'] = first['ATA'] - last['ATA']
        return diff[['ALSA', 'Δ ALSA', 'ATA', 'Δ ATA', 'Color', 'Rarity']]


    def get_top(self, column, count=10, asc=True, card_color=None, card_rarity=None, deck_color='', play_lim=None):
        frame = self.card_frame(deck_color=deck_color, summary=True, card_rarity=card_rarity)
        frame = frame.sort_values(column, ascending=asc)

        if card_color is not None:
            card_color = WUBRG.get_color_identity(card_color)
            frame = frame[frame['Color'] == card_color]

        if play_lim is not None:
            if type(play_lim) is float: play_lim *= self.get_games_played(deck_color)
            print(f'Minimum Games played to be included: {play_lim}')
            frame = frame[frame['# GP'] >= play_lim]

        return frame.head(count)

In [ ]:
class ArchFuncs:   
    def __init__(self, DATA):
        self._DATA = DATA
        
    def get_games_played(self, deck_color):
        if deck_color: return self._DATA.deck_archetype_frame(deck_color=deck_color, summary=True)['Games'].sum()
        else: return self._DATA.deck_group_frame(name='All Decks', summary=True)['Games']
    
    def get_avg_winrate(self, day=None, arch='All Decks'):
        if day: return self._DATA.deck_group_frame(date=day, summary=False).loc[(day, arch)]['Win %']
        else: return self._DATA.deck_group_frame(date=day, summary=True).loc[arch]['Win %']   
    
    def get_archetype_frame(self, colors, roll=None):
        if roll is None: roll = settings.ROLL
        win_rate_frame = self._DATA.deck_archetype_frame(deck_color=colors)
        win_rate_frame.index = [tup[0] for tup in win_rate_frame.index]
        #win_rate_frame = win_rate_frame[['Splash', 'Games', 'Win %']]
        win_rate_frame = win_rate_frame[win_rate_frame['Splash'] == False][['Wins', 'Games']]
        rolling = win_rate_frame.rolling(window=roll, min_periods=1, center=True).mean().round()
        rolling['Win %'] = round((rolling['Wins'] / rolling['Games']) * 100, 2)
        rolling['Avg. Win%'] = [self.get_avg_winrate(idx) for idx in win_rate_frame.index]
        rolling['2C Win%'] = [self.get_avg_winrate(idx, arch='Two-color') for idx in win_rate_frame.index]
        rolling['Win % Offset'] = rolling['Win %'] - rolling['Avg. Win%']
        return rolling

    def get_archetype_winrate_history(self, color_filter=None, roll=None):
        if roll is None: roll = settings.ROLL
        d = dict()
        for col in WUBRG.COLOR_PAIRS:
            temp_frame = self.get_archetype_frame(col)
            d[col] = temp_frame['Win %']
        d['AVG'] = temp_frame ['Avg. Win%']
        d['2C'] = temp_frame ['2C Win%']

        test_frame = pd.DataFrame.from_dict(d)
        test_frame.index = [idx[5:] for idx in test_frame.index]
        if color_filter:
            col_filt = [col for col in WUBRG.COLOR_PAIRS if color_filter in col] + ['AVG', '2C'] 
            test_frame = test_frame[col_filt]

        rolling = test_frame.rolling(window=roll, min_periods=1, center=True).mean()
        return rolling

    def plot_archetype_winrate_history(self, color_filter=None, roll=None):
        if roll is None: 
            roll = settings.ROLL
        test_frame = self.get_archetype_winrate_history(color_filter, roll)
        lc = LineColors()
        title = f"Archetype Winrates (from 17Lands)\n{self._DATA.SET} - {self._DATA.FORMAT}"

        col_filt = f"Color Filter: {color_filter}"
        rol_filt = f"Rolling Average: {roll} Days"

        if color_filter and roll > 1:
            title += f"\n{col_filt}  -  {rol_filt}"
        elif color_filter:
            title += f"\n{col_filt}"
        elif roll > 1:
            title += f"\n{rol_filt}"
        
        test_frame.plot(figsize=(20, 10), color=lc.get_col_array(color_filter), title=title, lw=2.5, grid=True)
        plt.xlabel("Date")
        plt.ylabel("Win Percent")
        
        
    #TODO: Implement a more generic version of this that takes in a list of deck colours to include as output. 
    def get_archetype_playrate_history(self, color_filter=None, roll=None):
        if roll is None: roll = settings.ROLL
        d = dict()
        for col in WUBRG.COLOR_PAIRS:
            d[col] = self.get_archetype_frame(col)['Games']

        test_frame = pd.DataFrame.from_dict(d)
        test_frame.index = [idx[5:] for idx in test_frame.index]
        rolling = test_frame.rolling(window=roll, min_periods=1, center=True).mean()
        total = rolling.sum(axis=1)
        playrate = rolling.divide(list(total),axis=0) * 100

        if color_filter:
            col_filt = [col for col in WUBRG.COLOR_PAIRS if color_filter in col]
            playrate = playrate[col_filt]

        return playrate

    def plot_archetype_playrate_history(self, color_filter=None, roll=None):
        if roll is None: roll = settings.ROLL
        test_frame = self.get_archetype_playrate_history(color_filter, roll)
        lc = LineColors()
        title = f"Archetpye Playrates (from 17Lands)\n{self._DATA.SET} - {self._DATA.FORMAT}"

        col_filt = f"Color Filter: {color_filter}"
        rol_filt = f"Rolling Average: {roll} Days"

        if color_filter and roll > 1:
            title += f"\n{col_filt}  -  {rol_filt}"
        elif color_filter:
                title += f"\n{col_filt}"
        elif roll > 1:
                title += f"\n{rol_filt}"
        test_frame.plot(figsize=(20, 10), color=lc.get_col_array(color_filter), title=title, lw=2.5, grid=True)
        plt.xlabel("Date")
        plt.ylabel("Percent of Metagame")

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

class PlotterHelper:
    def __init__(self, DATA):
        self._DATA = DATA
        self._FIG = None
        self._AX = None
        os.makedirs(self.get_folder_path(), exist_ok=True)
        
    @property
    def FIG(self):
        return self._FIG
    
    @property
    def AX(self):
        return self._AX
    
    def _get_sub_ax(self, x=None, y=None):
        if not isinstance(self.AX, np.ndarray):
            return self.AX
        elif self.AX.ndim == 1:
            return self.AX[x if x is not None else y]
        elif self.AX.ndim == 2:
            return self.AX[x, y]
        else:
            raise Excpetion('Unknown axes format.')
            
    
    def get_folder_path(self):
        return os.path.join(settings.GRAPH_DIR_LOC, settings.GRAPH_DIR_NAME, self._DATA.SET, self._DATA.FORMAT)
         
    
    def get_file_path(self, filename):
        return os.path.join(self.get_folder_path(), filename)
    
    
    def new_single_plot(self, title, width=8, height=6, fontsize=settings.TITLE_SIZE):
        self._FIG, self._AX = plt.subplots(1, 1)
        self.FIG.set_size_inches(width, height)
        self.AX.set_title(f"{self._DATA.SET} {self._DATA.SHORT_FORMAT}: {title}", fontsize=fontsize)
        self.set_x_axis_weekly()
        return self.FIG, self.AX
    
    
    def new_quad_plot(self, title, width=12, height=8, fontsize=settings.TITLE_SIZE):
        self._FIG, self._AX = plt.subplots(2, 2)
        self.FIG.set_size_inches(width, height)
        plt.figtext(0.4, 0.9, f"{self._DATA.SET} {self._DATA.SHORT_FORMAT}: {title}", fontsize=fontsize)
        self.set_x_axis_weekly(g_x=0, g_y=0)
        self.set_x_axis_weekly(g_x=0, g_y=1)
        self.set_x_axis_weekly(g_x=1, g_y=0)
        self.set_x_axis_weekly(g_x=1, g_y=1)
        return self.FIG, self.AX

    
    def accredit(self, y=0.01):
        plt.figtext(0.5, y, settings.ACCREDIT_STR, **settings.ACCREDIT_KWARGS)
        
        
    def desc_note(self, colors=None, roll=1, y=0.95):
        col_filt = f"Color Filter: {colors}"
        rol_filt = f"Rolling Average: {roll} Days"

        if colors and roll > 1:
            txt = f"{rol_filt}\n{col_filt}"
        elif colors: 
            txt = f"\n{col_filt}"
        elif roll > 1:
            txt = f"\n{rol_filt}"
        else:
            txt = ""
            
        plt.figtext(0.5, y, txt, **settings.FILTER_KWARGS)
        
    
    def set_labels(self, x_label="", y_label="", fontsize=settings.LABEL_SIZE, g_x=None, g_y=None):
        ax = self._get_sub_ax(g_x, g_y)
        ax.set_xlabel(x_label, fontsize=fontsize)
        ax.set_ylabel(y_label, fontsize=fontsize)
        
    
    def set_data(self, data, col_list, inv_y=False, inv_x=False, g_x=None, g_y=None):
        ax = self._get_sub_ax(g_x, g_y)
        # TODO: Add line_colors.
        ax.plot(data.index, data[col_list], label=col_list)
        if inv_y: ax.invert_yaxis()
        if inv_x: ax.invert_xaxis()
        ax.legend()
        
        
    def set_x_axis_weekly(self, g_x=None, g_y=None):
        ax = self._get_sub_ax(g_x, g_y)
        weeks = mdates.DayLocator(interval=7)
        ax.xaxis.set_major_locator(weeks)
        days = mdates.DayLocator(interval=1)
        ax.xaxis.set_minor_locator(days)
        
        
    def save_fig(self, filename, dpi=None):
        if dpi is None: dpi = settings.DPI
        self.FIG.savefig(self.get_file_path(filename.replace(' ', '_')), dpi=dpi)


In [ ]:
class SingleCardFuncs:   
    def __init__(self, DATA):
        self._DATA = DATA
        os.makedirs(self.get_folder_path(), exist_ok=True)
        
    def get_folder_path(self):
        return os.path.join(settings.GRAPH_DIR_LOC, settings.GRAPH_DIR_NAME, self._DATA.SET, self._DATA.FORMAT)
         
    def get_file_path(self, filename):
        return os.path.join(self.get_folder_path(), filename)

    def get_card_summary(self, card_name, colors='', roll=None):
        if roll is None: roll = settings.ROLL
        frame = self._DATA.card_frame(name=card_name, deck_color=colors)[['GIH WR', 'GND WR', 'ATA', 'ALSA', '# GP', '# GIH', '# Picked', '# Seen']]
        frame.index = [tup[0][5:] for tup in frame.index]
        rolling = frame.rolling(window=roll, min_periods=1, center=True).mean()
        return rolling

    def plot_card_summary(self, card_name, colors='', roll=None):
        if roll is None: roll = settings.ROLL
        colors = WUBRG.get_color_identity(colors)
        rolling = self.get_card_summary(card_name, colors, roll)
        
        plot_help = PlotterHelper(self._DATA)
        fig, ax = plot_help.new_quad_plot(card_name)
        plot_help.accredit()
        plot_help.desc_note(colors=colors, roll=roll)
                
        plot_help.set_labels(y_label="Win Percent", g_x=0, g_y=0)
        plot_help.set_data(rolling, ['GIH WR', 'GND WR'], g_x=0, g_y=0)
        
        plot_help.set_labels(y_label="Pick Number", g_x=0, g_y=1)
        plot_help.set_data(rolling, ['ATA', 'ALSA'], inv_y=True, g_x=0, g_y=1)
        
        plot_help.set_labels(x_label="Date", y_label="# of Games", g_x=1, g_y=0)
        plot_help.set_data(rolling, ['# GP', '# GIH'], g_x=1, g_y=0)
        
        plot_help.set_labels(x_label="Date", y_label="# of Cards", g_x=1, g_y=1)
        plot_help.set_data(rolling, ['# Picked', '# Seen'], g_x=1, g_y=1)
        
        
        plot_help.save_fig(f"pcs_{card_name}_{colors}.png")
      
    
    def get_pick_stats(self, card_name, roll=None):
        if roll is None: roll = settings.ROLL
        taken_data = self._DATA.card_frame(name=card_name, deck_color='')[['ALSA', 'ATA']]
        taken_data.index = [tup[0][5:] for tup in taken_data.index]
        taken_data = taken_data.rolling(window=roll, min_periods=1, center=True).mean()
        return taken_data
    
    
    def plot_pick_stats(self, card_name, roll=None):
        if roll is None: roll = settings.ROLL
        taken_data = self.get_pick_stats(card_name, roll)
        
        plot_help = PlotterHelper(self._DATA)
        fig, ax = plot_help.new_single_plot(card_name)
        plot_help.accredit()
        plot_help.desc_note(roll=roll)
        
        plot_help.set_labels(x_label="Date", y_label="Pick Number")
        plot_help.set_data(taken_data, ['ATA', 'ALSA'], inv_y=True)
        
        plot_help.save_fig(f"pps_{card_name}.png")


    def card_archetype_performance(self, card_name):
        d = dict()
        d['AVG'] = self._DATA.card_frame(card_name, '', summary=True)
        for col in WUBRG.COLOR_PAIRS:
            d[col] = self._DATA.card_frame(card_name, col, summary=True)

        test_frame = pd.DataFrame.from_dict(d).T
        return test_frame
    
    
    def stat_archetype_performance(self, stat_name, color_cols=None):
        series = self._DATA.card_frame(summary=True)[stat_name]
        frame = series.reset_index(level=0)
        ret = pd.pivot_table(frame, index='Name', columns='Deck Colors')
        ret.columns = ret.columns.droplevel(0)
        ret = ret[WUBRG.COLOR_GROUPS]  #Re-orders the columns in WUBRG order
        if color_cols is not None:
            ret = ret[color_cols]
        return ret

In [ ]:
class SetManager:
    def __init__(self, SET):
        self._SET = SET
        self._DATA = {f: FramedDataFuncs(SET, f) for f in FORMATS}
        self._SET_METADATA = SetMetadata.get_metadata(SET)
    
    
    def check_for_updates(self):
        """Populates and updates all data properties, filling in missing data."""
        self.BO1.check_for_updates()   
        #self.BO3.check_for_updates()   
        #self.QD.check_for_updates()

    def reload_data(self):
        """Populates and updates all data properties, reloading all data."""
        self.BO1.reload_data()   
        self.BO3.reload_data()   
        self.QD.reload_data()  
    
    @property
    def SET(self):
        """The draft set."""
        return self._SET
            
    @property
    def BO1(self):
        """Premier Draft data."""
        return self._DATA['PremierDraft']
    
    @property
    def BO3(self):
        """Traditional Draft data."""
        return self._DATA['TradDraft']
    
    @property
    def QD(self):
        """Quick Draft data."""
        return self._DATA['QuickDraft']
    
    @property
    def SET_METADATA(self):
        """The metadata of the set."""
        return self._SET_METADATA
    
    @property
    def CARDS(self):
        """The list of cards in the set."""
        return self._SET_METADATA.CARD_LIST

# Initialization

In [ ]:
set_data = SetManager('NEO')

In [ ]:
set_data.check_for_updates()

In [ ]:
set_data.BO1.plot_card_summary('Imperial Oath')

In [ ]:
set_data.BO1.plot_pick_stats('Imperial Oath')

In [ ]:
raise Exception('Stopping Auto-Run!')

## Current Tests

In [ ]:
def gt(x, y):  #Used for function pointer shenanigans
    return x > y

def lt(x, y):  #Used for function pointer shenanigans
    return x < y

def filter_quadrant_cards_df(func1, func2, iwd_thresh=0, play_lim=0.01, card_rarity=None, deck_color=''):
    frame = set_data.BO1.card_frame(deck_color=deck_color, summary=True, card_rarity=card_rarity)
    
    # TODO: Make the mean different based on card rarity so cards aren't moved into incorrect categories.
    
    if play_lim is not None:
        if type(play_lim) is float: play_lim *= set_data.BO1.get_games_played(deck_color)
        print(f'Minimum Games played to be included: {play_lim}')
        frame = frame[frame['# GP'] >= play_lim]
        
    games_played_mean = frame['# GP'].mean()
    cards = frame[func1(frame['IWD'], iwd_thresh)]
    cards = cards[func2(cards['# GP'], games_played_mean)]
    cards = cards.sort_values('IWD', ascending=func1==lt)
    return cards


def get_trap_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_quadrant_cards_df(lt, gt, iwd_thresh, play_lim, card_rarity, deck_color)

def get_niche_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_quadrant_cards_df(gt, lt, iwd_thresh, play_lim, card_rarity, deck_color)

def get_staple_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_quadrant_cards_df(gt, gt, iwd_thresh, play_lim, card_rarity, deck_color)

def get_dreg_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_quadrant_cards_df(lt, lt, iwd_thresh, play_lim, card_rarity, deck_color)

In [ ]:
get_niche_cards(deck_color='GW', card_rarity='CU')

In [ ]:
set_data.BO1.card_archetype_performance("Era of Enlightenment")

# TODO

- Calculate archetype openess
 - GIH WR & ALSA based
 - 2.25 of a common per draft 
- Improve graphing capabilities
 - Modify Graphs so they also save an image to a cache folder.
 - Implement better graph titles and axes
 - Update functions to be more general, and have DataFrames piped into them.
- Move sets of files into subfolders based on functionality and relationship (DataFetch, Cards, Graphing, Utils, etc.)

# Data Graphing and Display

## Card Summary

In [ ]:
set_data.BO1.plot_card_summary('Imperial Oath')

In [ ]:
set_data.BO1.plot_card_summary("Befriending the Moths", roll=3)

In [ ]:
set_data.BO1.plot_card_summary("Michiko's Reign of Truth", roll=3)

In [ ]:
set_data.BO1.plot_card_summary("The Fall of Lord Konda", roll=3)

In [ ]:
set_data.BO1.plot_card_summary('Sunblade Samurai', roll=3)

In [ ]:
set_data.BO1.get_top('GIH WR', count=25, asc=False, deck_color='', card_color=None, card_rarity='RM', play_lim=0.005)

## Card Pick Order Stats

In [ ]:
set_data.BO1.plot_pick_stats('Imperial Oath')

In [ ]:
set_data.BO1.plot_pick_stats('Behold the Unspeakable', 3)

In [ ]:
to_graph = ['Imperial Oath', 'Behold the Unspeakable', 'Virus Beetle', 'Network Disruptor', 'Kumano Faces Kakkazan', 'Iron Apprentice']
for card in to_graph:
    set_data.BO1.plot_pick_stats(card, 3)

## Archetype Winrate History

In [ ]:
set_data.BO1.get_archetype_winrate_history()

In [ ]:
set_data.BO1.plot_archetype_winrate_history('')

In [ ]:
set_data.BO3.plot_archetype_winrate_history('')

## Archetype Playrate History

In [ ]:
set_data.BO1.get_archetype_playrate_history()

In [ ]:
set_data.BO1.plot_archetype_playrate_history('')

In [ ]:
set_data.BO3.plot_archetype_playrate_history('')

## Card Pick Order Changes

In [ ]:
diff = set_data.BO1.compare_card_evaluations('2022-02-10', '2022-03-27')
commons = diff[diff['Rarity'] == 'C']
uncommons = diff[diff['Rarity'] == 'U']

In [ ]:
SRT_TRG = 'Δ ATA'
commons.sort_values(SRT_TRG, ascending=False).head(20)

In [ ]:
commons.sort_values(SRT_TRG, ascending=True).head(20)

In [ ]:
uncommons.sort_values(SRT_TRG, ascending=False).head(10)

In [ ]:
uncommons.sort_values(SRT_TRG, ascending=True).head(10)